# Validation la qualité du dataframe créé

In [1]:
%config Completer.use_jedi = False

In [24]:
import sys 
sys.path.append('/home/cdsw/code')
import imp
from utils import *
import numpy as np
import math
import openpyxl
import pandas as pd
import dateutil
from dateutil.parser import parse
import warnings
warnings.filterwarnings("ignore")
from Prev.multiple_sheet_process import * 
from Prev.single_sheet_process import *

In [25]:
import Prev.multiple_sheet_process
import Prev.single_sheet_process
import utils
imp.reload(utils)
imp.reload(Prev.multiple_sheet_process)
imp.reload(Prev.single_sheet_process)
from utils import *
from Prev.multiple_sheet_process import * 
from Prev.single_sheet_process import * 

In [26]:
pd.set_option("max_rows", 5000)
pd.set_option('max_columns', 50)

In [27]:
emplacement = "/mnt/Plateforme/1 - Collecte des données sources auprès des OP/19 - Tresorerie/Prévisions DEA mensualisées/ajout_plateforme"
extension = "*.xlsx"
list_filename = get_files_in_directory(emplacement,extension)


In [28]:
normal_values_nature_montant = ['Autres dépenses allocations','Impositions de toutes natures', '     dont prélèvement PAS',
 'Pôle emploi - Allocataires','     dont reversement PAS','Pour dépenses Unédic (part tiers)', 'ASP - Activité partielle',
 'Dépenses autres publics','ARE / AREF','    Aides','Cumul inversé (à fin de mois)','Caisses de retraite complémentaire',
 'Total dépenses', 'Participation entreprise CSP / CRP','Somme retraitée','Raisons','Pôle emploi - CSP', 'Autres produits',
 'Acoss - CEA (chèque emploi-associatif)', '    Allocations brutes','Pôle emploi - PAS allocataires',
 'Allocation partielle de longue durée (APLD)', 'Autres (accompagnement CSP, recouvrements)', 'Somme autres',
 'Total dépenses - Equilibre technique','Solde = recettes - dépeneses', 'Variation de tréso','Caisse de retraites net',
 'Contributions','  dont surplus de dépenses crise* ','Acoss - TREC (part patronale)','dont intérêt', 'Dépenses ARE AREF ASP ',
 'Éléments exceptionnels', '    Prélèvements sociaux','Autres aides (adr, aide fin de droit, ...)', 'ATI','11% Pôle Emploi',
 'Recettes yc crise','Pôle emploi - Participation Unédic (11%)', 'Ajustement', 'Gestion administrative',
 'Variation de trésorerie',"Remboursements d'indus",'Activité partielle','Somme','dont Pole emploi', 'AREP',
 'Autres (AS-FNE, Plan rebond, ...)','Indus, avances et acomptes','Recettes techniques','Cumul','     dont reversement secu',
 'Contributions et autres produits','Calculé', "En millions d'euros",'Autres allocations (décès, ...)',
 "       dont manque à gagner Activité partielle + garde d'enfant","Contributions d'Assurance chômage",'Total',
 'Autres', '    Prélèvements sociaux et fiscaux','Somme brute','Variation de trésorerie - Equilibre technique',
 'Caisses de retraite complémentaire (dont précomptes)','Dépenses yc crise','    Reversements et compléments',
 'Intérêts', 'Retraitements','Acoss - EPM (particuliers employeurs)','Sécurité sociale (CSG, CRDS, maladie)',
 'CRP / CTP / CSP, y compris équivalents ARE','Total recettes','ARE / AREF, y compris UE','Somme dépenses techniques',
 'CRP / CSP, y compris équivalents ARE','  dont prélèvement','Frais de gestion et décaissements sur immobilisations',
 'Financement et frais de gestion opérateurs','Autres produits (gestion administrative et financière)',
 'Pôle emploi - URSAFF allocataires','Pour dépenses Unédic (dépenses allocataires)',
 '10% Pôle Emploi','Intérêts nets sur emprunts','Frais de fonctionnement',
 'Autres dépenses','Autres aides  (prime CSP, aide fin de droit, ...)',
 "Contributions et autres recettes d'Assurance chômage",'Revenus financiers', 'dont frais de fonctionnement et de gestion',
 'Prélèvement à la source','Caisses de retraites complémentaire (dont précomptes)',
 '    dont prélèvement sécu', 'Dépenses allocataires','Pôle emploi - Contribution (11%)', 'Total Dépenses',
 'Autres aides (prime CSP, aide fin de droit, ...)','Equilibre technique','Frais de fonctionnement et de gestion',
 'Autres produits, conventions diverses','Dépenses techniques',
 'Eléments exceptionnels','Etat - Prélèvement à la source',"Aides à la reprise et création d'entreprise",'Contributions principales',
 'Activité partielle, allocation partielle de longue durée (APLD)','Intérêts sur emprunts',
 'Conventions diverses, y compris UE', 'Impositions de toutes natures (CSG)',
 'Caisses de retraites complémentaire', '  dont Reversement hors effet crise', 'Pour recettes Unédic','Somme cumulée',
 'Caisses de retraite allocataires', 'Raison','       dont décalage de trésorerie',
 'Total recettes - Equilibre technique','Acoss - Réduction générale','Acoss - CSG activité' ]


In [23]:


def data_quality(df,normal_values_nature_montant):
    """  
    Cette fonction permet de vérifier le résultat de la transformation des fichiers excel
    params: 
    - df: dataframe à vérifier 
    """
    clean_data = dict()
    
    # vérifier la colonne mois_nom
    normal_values_mois_nom = ['total', 'janvier','février', 'mars', 'avril', 'mai', 'juin', 'juillet', 'août', 'aout',
                              'septembre', 'octobre', 'novembre', 'décembre']
    
    somme = df['mois_nom'].isin(normal_values_mois_nom).sum()
    clean_data['mois_nom'] = (somme == len(df))
    
    # vérifier la colonne type montant 
    somme= df['type_montant'].str.contains('réalisé', case = False).sum() +\
    df['type_montant'].str.contains('estimé', case = False).sum() +\
    df['type_montant'].str.contains('prévu', case = False).sum() + \
    df['type_montant'].str.contains('observé', case = False).sum() 
    clean_data['type montant'] = (somme == len(df))

    
    # vérifier la colonne nature montant 
    
    somme = df['nature_montant'].isin(normal_values_nature_montant).sum()
    clean_data['nature montant']  = (somme == len(df))
    
    clean_data_ = True
    for e in list(list(clean_data.values())):
        clean_data_ = clean_data_ * e 
    
    return clean_data, clean_data_

In [186]:

for year in df: 
    print("------------------", year,"------------------")
    print(data_quality(df[year], normal_values_nature_montant))

------------------ 2011 ------------------
({'mois_nom': False, 'type montant': False, 'nature montant': True}, False)
------------------ 2012 ------------------
({'mois_nom': True, 'type montant': True, 'nature montant': True}, True)
------------------ 2013 ------------------
({'mois_nom': True, 'type montant': True, 'nature montant': True}, True)
------------------ 2014 ------------------
({'mois_nom': True, 'type montant': True, 'nature montant': True}, True)
------------------ 2015 ------------------
({'mois_nom': True, 'type montant': True, 'nature montant': True}, True)
------------------ 2016 ------------------
({'mois_nom': True, 'type montant': True, 'nature montant': True}, True)
------------------ 2017 ------------------
({'mois_nom': True, 'type montant': True, 'nature montant': True}, True)
------------------ 2018 ------------------
({'mois_nom': True, 'type montant': True, 'nature montant': True}, True)
------------------ 2019 ------------------
({'mois_nom': True, 'type 

In [17]:
filename = list_filename[-5]
prevision_data_ingestion_single_sheet(filename, extension, normal_values_nature_montant)

durée de traitement = 0:00:01.079229


,version_prev,annee,mois,mois_nom,nature_montant,type_montant,montant_millions
0,2021-02,2020,1,janvier,Contributions d'Assurance chômage,Observé,3655.348133
1,2021-02,2020,1,janvier,Contributions principales,Observé,2395.795352
2,2021-02,2020,1,janvier,Impositions de toutes natures,Observé,1259.552781
3,2021-02,2020,1,janvier,"Conventions diverses, y compris UE",Observé,32.004273
4,2021-02,2020,1,janvier,Participation entreprise CSP / CRP,Observé,34.625761
5,2021-02,2020,1,janvier,Autres produits,Observé,0.000000
6,2021-02,2020,1,janvier,Total recettes,Observé,3721.978167
8,2021-02,2020,1,janvier,Dépenses allocataires,Observé,3800.452507
9,2021-02,2020,1,janvier,Allocations brutes,Observé,3055.949747
10,2021-02,2020,1,janvier,ARE / AREF,Observé,3062.045231
